# Setup and run Bayesian inference on a GW event

## Imports and setup

In [3]:
import os
import sys
import shutil
import glob
import h5py
import subprocess

import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex = True)

plt.rcParams.update(  # try to match font sizes of document
  {'axes.labelsize': 20,
   'axes.titlesize': 20,
   'legend.fontsize': 20,
   'xtick.labelsize': 20,
   'ytick.labelsize': 20,
   'text.usetex': True,
   'font.family': 'serif',
   'font.serif': ['palatino'],
   'savefig.dpi': 300
   })

%pylab inline

from glue.ligolw import ligolw
from glue.ligolw import table
from glue.ligolw import lsctables
from glue.ligolw import ilwd
from glue.ligolw import utils as ligolw_utils

import pycbc.strain
import pycbc.psd
from pycbc.pnutils import mass1_mass2_to_mchirp_eta
from pycbc.waveform import td_approximants, fd_approximants
from pycbc.waveform import get_two_pol_waveform_filter, get_td_waveform
from pycbc import DYN_RANGE_FAC
from pycbc.types import FrequencySeries, zeros
from pycbc.filter import match, overlap, sigma, make_frequency_series
from pycbc.scheme import CPUScheme, CUDAScheme
from pycbc import pnutils
from pycbc import frame

import GWNRTools
from GWNRTools.Utils.SupportFunctions import make_padded_frequency_series
from GWNRTools.DataAnalysis import get_unique_hex_tag
import GWNRTools.DataAnalysis as DA

sys.path.append('/home/prayush.kumar/src/GWNRTools/bin/')
sys.path.append('/home/prayush/src/GWNRTools/bin/')
#sys.path.append('/home/prayush.kumar/local/venv/pycbc_master_enigma/src/GWNRTools/bin/')

Populating the interactive namespace from numpy and matplotlib


/home/prayush/local/venv/pycbc_inf/local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['rc']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Could not import ligolw in /home/prayush/src/GWNRTools/GWNRTools/Stats/FisherMatrixUtilities.pyc, LIGO XML tables wont be read


In [6]:
run_dir = '/home/prayush/research/test_pycbc_gw10914'
try: os.makedirs(run_dir)
except: pass
os.chdir(run_dir)

In [8]:
!pwd

/home/prayush/research/test_pycbc_gw10914


In [23]:
!rm -f *

In [24]:
!ls

## Generate sampler / inference config files

In [1]:
!gwnrtools_write_injection_inference_configs -h

Could not import ligolw in /home/prayush/src/GWNRTools/GWNRTools/Stats/FisherMatrixUtilities.pyc, LIGO XML tables wont be read
usage: /home/prayush/src/GWNRTools/bin//gwnrtools_write_injection_inference_configs [--options]

Get and write configuration files for generating a workflow to perform
Bayesian parameter estimation runs on a custom set of simulated signals

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --write-data-config WRITE_DATA_CONFIG
                        Write data config files and exit.
  --write-sampler-config WRITE_SAMPLER_CONFIG
                        Write sampler config files and exit.
  --write-inference-config WRITE_INFERENCE_CONFIG
                        Write inference config files and exit.
  --show-available-configs
                        Show available options for all configurations.
  --output-dir OUTPUT_DIR
              

In [10]:
!gwnrtools_write_injection_inference_configs --show-available-configs

Could not import ligolw in /home/prayush/src/GWNRTools/GWNRTools/Stats/FisherMatrixUtilities.pyc, LIGO XML tables wont be read
inference: ['gw150914_like']
data: ['gw150914-like-gaussian']
sampler: ['cpnest_v1', 'emcee_pt_v1']


In [33]:
!gwnrtools_write_injection_inference_configs --verbose\
  --write-sampler-config emcee_pt_v1 --write-inference-config gw150914_like

Could not import ligolw in /home/prayush/src/GWNRTools/GWNRTools/Stats/FisherMatrixUtilities.pyc, LIGO XML tables wont be read
2020-03-03 23:55:45,407 Writing config file for sampler settings..
2020-03-03 23:55:45,408 Writing config file for inference settings..
2020-03-03 23:55:45,408 Done


In [34]:
!ls

emcee_pt_v1.ini  gw150914_like.ini


In [35]:
!cat emcee_pt_v1.ini gw150914_like.ini

[sampler]
name = emcee_pt
nwalkers = 500
ntemps = 20
effective-nsamples = 4000
checkpoint-interval = 2000
max-samples-per-chain = 1000

[sampler-burn_in]
burn-in-test = nacl & max_posterior

;
;   Sampling transforms
;
[sampling_params]
; parameters on the left will be sampled in
; parametes on the right
mass1, mass2 : mchirp, q

[sampling_transforms-mchirp+q]
; inputs mass1, mass2
; outputs mchirp, q
name = mass1_mass2_to_mchirp_q
[model]
name = gaussian_noise
low-frequency-cutoff = 20.0

[variable_params]
; waveform parameters that will vary in MCMC
delta_tc =
mass1 =
mass2 =
spin1_a =
spin1_azimuthal =
spin1_polar =
spin2_a =
spin2_azimuthal =
spin2_polar =
distance =
coa_phase =
inclination =
polarization =
ra =
dec =

[static_params]
; waveform parameters that will not change in MCMC
approximant = IMRPhenomPv2
f_lower = 20
f_ref = 20
; we'll set the tc by using the trigger time in the data
; section of the config file + delta_tc
trigger_time = ${data|trigger-time}

[prior-delta_tc

## Write workflow config

In [37]:
out_str = '''\
'''

In [ ]:
with open('config.ini', 'w') as fout:
    fout.write(out_str)

## Generate workflow

In [36]:
!gwnrtools_create_public_event_inference_workflow -h

Could not import ligolw in /home/prayush/src/GWNRTools/GWNRTools/Stats/FisherMatrixUtilities.pyc, LIGO XML tables wont be read
usage: /home/prayush/src/GWNRTools/bin//gwnrtools_create_public_event_inference_workflow [--options]

Setup workflow to perform Bayesian parameter estimation runs on a custom set
of public gravitational-wave events using open data

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --output-dir OUTPUT_DIR
                        Output directory path.
  --force               If the output-dir already exists, overwrite it.
                        Otherwise, an OSError is raised.
  --save-backup         Don't delete the backup file after the run has
                        completed.
  --nprocesses NPROCESSES
                        Number of processes to use. If not given then only a
                        single core will be used.
  --

## Submit & monitor workflow